## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [1]:
# imports
import torch #导包
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #设备

In [2]:
# download the data (uncomment if to download the data locally)
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip #下载数据
#!unzip hymenoptera_data.zip #解压文件

In [3]:
# create data loaders

data_dir = 'hymenoptera_data' #数据文件

# custom transformer to flatten the image tensors
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size #初始化

    def __call__(self, img):
        result = torch.reshape(img, self.new_size) #重塑图片大小
        return result

# transformations used to standardize and normalize the datasets
data_transforms = { #标准化规范化图像
    'train': transforms.Compose([ #组合操作
        transforms.Resize(224), #把图片大小转换成224
        transforms.CenterCrop(224), #从图片中心剪裁大小为224*224的
        transforms.ToTensor(), #将图片转化为张量
        ReshapeTransform((-1,)) # flattens the data
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,)) # flattens the data
    ]),
}

# load the correspoding folders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), #从相应文件夹中加载图像
                                          data_transforms[x])
                  for x in ['train', 'val']}

# load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size=len(image_datasets['train']),
                                            shuffle=True) #加载训练数据集

test_dataset = torch.utils.data.DataLoader(image_datasets['val'],
                                           batch_size=len(image_datasets['val']),
                                           shuffle=True) #加载测试数据集

In [4]:
# build the LR model
class LR(nn.Module): #逻辑回归模型
    def __init__(self, dim): #初始化
        super(LR, self).__init__() #调用父类的初始化方法
        self.linear = nn.Linear(dim, 1)
        nn.init.zeros_(self.linear.weight) #权值置0
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        x = self.linear(x) #对x做线性变换
        x = torch.sigmoid(x) #sigmoid函数
        return x 

In [5]:
# predict function
def predict(yhat, y): #预测
    yhat = yhat.squeeze() #减少维度
    y = y.unsqueeze(0) #在第一维上增加维度
    y_prediction = torch.zeros(y.size()[1]) #预测结果置0
    for i in range(yhat.shape[0]):
        if yhat[i] <= 0.5:
            y_prediction[i] = 0
        else:
            y_prediction[i] = 1
    return 100 - torch.mean(torch.abs(y_prediction - y)) * 100 #预测结果

In [8]:
# model config
dim = train_dataset.dataset[0][0].shape[0] #获得数据维度
lrmodel = LR(dim).to(device)
criterion = nn.BCELoss() #对数据做二元交叉熵并且求平均
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001) #构建优化器

In [8]:
# training the model
costs = [] #损失

for ITER in range(100):
    lrmodel.train() #启用BatchNormalization和 Dropout，将BatchNormalization和Dropout置为True
    x, y = next(iter(train_dataset)) #获取训练数据集
    test_x, test_y = next(iter(test_dataset)) #获取测试数据集

    # forward
    yhat = lrmodel.forward(x.to(device)) #正向传播

    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor).to(device)) #计算损失
    train_pred = predict(yhat, y) #预测训练结果

    # backward
    optimizer.zero_grad() #把梯度置0
    cost.backward() #计算梯度
    optimizer.step() #更新模型参数
    
    # evaluate
    lrmodel.eval() #不启用 BatchNormalization 和 Dropout，将BatchNormalization和Dropout置为False
    with torch.no_grad(): #torch.no_grad() 是一个上下文管理器
        yhat_test = lrmodel.forward(test_x.to(device))
        test_pred = predict(yhat_test, test_y) #预测测试数据

    if ITER % 10 == 0:
        costs.append(cost) #记录损失

    if ITER % 10 == 0:
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER, 
                                                                                    cost, 
                                                                                    train_pred,
                                                                                    test_pred)) #输出损失数据
   

Cost after iteration 0: 0.6931471228599548 | Train Acc: 50.40983581542969 | Test Acc: 45.75163269042969
Cost after iteration 10: 0.6691471338272095 | Train Acc: 64.3442611694336 | Test Acc: 54.24836730957031
Cost after iteration 20: 0.6513182520866394 | Train Acc: 68.44261932373047 | Test Acc: 54.24836730957031
Cost after iteration 30: 0.6367825269699097 | Train Acc: 68.03278350830078 | Test Acc: 54.24836730957031
Cost after iteration 40: 0.6245337128639221 | Train Acc: 69.67213439941406 | Test Acc: 54.90196228027344
Cost after iteration 50: 0.6139225363731384 | Train Acc: 70.90164184570312 | Test Acc: 56.20914840698242
Cost after iteration 60: 0.6045235395431519 | Train Acc: 72.54098510742188 | Test Acc: 56.86274337768555
Cost after iteration 70: 0.5960512161254883 | Train Acc: 74.18032836914062 | Test Acc: 57.51633834838867
Cost after iteration 80: 0.5883084535598755 | Train Acc: 73.77049255371094 | Test Acc: 57.51633834838867
Cost after iteration 90: 0.5811557769775391 | Train Acc: 

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)